# テーブル概要
accident:  
event:  
gv(general vehicle):車両一般  
ve(Exterior Vehicle):車両外部  
vi(Interior Vehicle):車両内部  
oa(OCCUPANT ASSESSMENT):乗員の調査  
oi(OCCUPANT INJURY):乗員の傷害(mergeに使用できるkeyの値が同一でも傷害箇所によってレコードが増加)  

# 最終的な作成データ
- Crash year 2010–2015
- Vehicle model year 2001–2015
- Light vehicles (passenger cars, pick-ups and mini-vans) 
- Non-ejected occupants
- Occupant age 15 or higher
- Occupants with known injury status or fatality

# ライブラリのインポート Pandasの表示設定
同一cellに複数テーブルを表示  
全カラムを表示  
最大表示行数:500  
1つのカラムの最大表示文字数:200  
floatの有効桁数:4  
色付き文字の出力:print(pycolor.RED + '文字列' + pycolor.END)  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import sys
from IPython import embed
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 200)
pd.options.display.float_format = '{:.4g}'.format
class pycolor:
    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    PURPLE = '\033[35m'
    CYAN = '\033[36m'
    WHITE = '\033[37m'
    END = '\033[0m'
    BOLD = '\038[1m'
    UNDERLINE = '\033[4m'
    INVISIBLE = '\033[08m'
    REVERCE = '\033[07m'

# NASS CDSデータの読み込み

In [2]:
path = os.path.dirname(os.path.abspath('__file__'))
file_name = ['accident', 'event', 'gv', 'oa', 'oi', 've', 'vi']#, 'makmod'
cds_key = []
cds = {}
for year in range(2010, 2016):
    for file in file_name:
        df = pd.read_sas(os.path.join(path, str(year), 'FormattedData', '{}.sas7bdat'.format(file)))
        cds_key.append('{}_{}'.format(file, year - 2000))
        cds['{}_{}'.format(file, year - 2000)] = df

In [3]:
path = os.path.dirname(os.path.abspath('__file__'))
expand_name = ['airbag', 'bagseat', 'childseat', 'seatloc', 'tire', 'tiredmg']
exp_key = []
exp = {}
for year in range(2011, 2016):
    for expand in expand_name:
        try:
            df = pd.read_sas(os.path.join(path, str(year), 'ExpandedSAS', '{}.sas7bdat'.format(expand)))
        except FileNotFoundError:
            df = pd.read_sas(os.path.join(path, str(year),  'ExpandedSAS', 'UNFORMATTED', '{}.sas7bdat'.format(expand)))
        exp_key.append('{}_{}'.format(expand, year - 2000))
        exp['{}_{}'.format(expand, year - 2000)] = df

# 特定のカラムが存在するか確認
引数：DFを格納した辞書, 有無を確かめたいカラムのリスト, オプション:調べたいDFのkey(デフォルトは全てのDF)

In [4]:
def exist_check(df_dic, check_columns, df_keys = None):
    if df_keys is None:
        df_keys = df_dic.keys()
    return_df = pd.DataFrame()
    for key in df_keys:
        df_columns = df_dic[key].columns.values
        check_dic = {}
        for check_column in check_columns:
            check_flag = 0
            for df_column in df_columns:
                if df_column == check_column:
                    check_flag = 1
            check_dic[check_column] = check_flag
        check_df = pd.io.json.json_normalize(check_dic)
        check_df = check_df.rename(index = {0: key})
        return_df = pd.concat([return_df, check_df])
    return_df.head(len(return_df))
    return return_df

# 同名のDFで，年毎に異なるカラムを持つか確認
年によって異なるカラムを持つファイル名とそのカラム名を出力する

In [5]:
def year_check(df_dic):
    df_keys = df_dic.keys()
    return_dic = {}
    name_list = []
    for key in df_keys:
        split_key = key.split('_')
        name = split_key[0]
        year = split_key[1]
        df_columns = df_dic[key].columns.values.tolist()
        if name not in name_list:
            name_list.append(name)
            return_dic[name] = pd.DataFrame()
        for column in df_columns:
            return_dic[name].loc[year, column] = 1
    for name in name_list:
        if return_dic[name].isnull().values.sum() != 0:
            print(pycolor.RED + name + pycolor.END)
            column_list = return_dic[name].columns.values.tolist()
            column_series = return_dic[name].isnull().any()
            for column in column_list:
                if column_series[column] == True:
                    print(column)
    return return_dic

#  テーブル間で重複するカラムの出力
デフォルトで重複は15年について調べる

In [6]:
def same_column(df_dic, conf_year = '15'):
    df_keys = df_dic.keys() 
    column_dic = {}
    name_list = []
    duplicate_list = []
    return_df = pd.DataFrame()
    for key in df_keys:
        name = key.split('_')[0]
        year = key.split('_')[1]
        if year == conf_year:
            column_dic[name] = df_dic[key].columns.values.tolist()
            name_list.append(name)
    for name in name_list:
        duplicate_list.extend(column_dic[name])
    duplicate_list = [x for x in set(duplicate_list) if duplicate_list.count(x) > 1]
    for name in name_list:
        for column in column_dic[name]:
            if column in duplicate_list:
                return_df.loc[name, column] = 1
    return return_df

# 重複したレコードの抽出
引数：DF辞書, DFのkey, 重複判定column(リスト)   
重複判定columnを先頭にしたDFを出力  

In [7]:
def duplicate(df_dic, dic_key, df_key):
    df = df_dic[dic_key]
    df = df[df.duplicated(subset = df_key, keep = False)]
    columns = list(df.columns.values)
    for column in df_key:
        columns.remove(column)
    return df.loc[:, df_key + columns]

In [8]:
cds_column = year_check(cds)

oa
CHOWUSED
LATCHDES
LATCHUSE
POSPRES
POSUSE
POSGUIDE
ve
TOWRES


In [9]:
cds_column['oa'][['CHOWUSED', 'LATCHDES', 'LATCHUSE', 'POSPRES', 'POSUSE', 'POSGUIDE']]
cds_column['ve'][['TOWRES']]

,CHOWUSED,LATCHDES,LATCHUSE,POSPRES,POSUSE,POSGUIDE
10,nan,nan,nan,nan,nan,nan
11,1,nan,nan,nan,nan,nan
12,1,1,1,1,1,1
13,1,1,1,1,1,1
14,1,1,1,1,1,1
15,1,1,1,1,1,1


,TOWRES
10,1
11,nan
12,nan
13,nan
14,nan
15,nan


In [10]:
cds['accident_11'].head()

,AAIS,AAIS08,AINJSER,AINJSER8,AINJURED,AINJURD8,ALCINV,ATREAT,CASEID,CASENO,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSU,PSUSTRAT,RATWGT,STRATIF,TIME,VEHFORMS,YEAR,VERSION,ADMINSS
0,2.254e-314,2.375e-314,2.375e-314,2.375e-314,2.265e-314,2.353e-314,2,2.265e-314,b' 1H',1,2,2,2,2.265e-314,1,2,3,322.8,b'H',2041,1,2011,24,0
1,nan,nan,nan,nan,nan,nan,2,nan,b' 2D',2,3,2,2,2.265e-314,1,2,3,76.81,b'D',300,1,2011,24,0
2,nan,nan,nan,nan,nan,nan,2,nan,b' 3F',3,6,2,1,2.265e-314,1,2,3,259.4,b'F',1221,1,2011,24,0
3,nan,nan,nan,nan,nan,nan,2,nan,b' 4H',4,5,2,1,2.265e-314,1,2,3,407.7,b'H',745,1,2011,24,0
4,1,1,2.259e-314,2.375e-314,1,1,2,3,b' 5K',5,5,2,3,6,1,2,3,42.13,b'K',1035,2,2011,24,0


In [11]:
cds['event_11'].head()

,ACCSEQ,CASEID,CASENO,CLASS2,CLASS1,GADEV1,GADEV2,OBJCONT,PSU,RATWGT,STRATIF,VEHNUM,VERSION
0,1,b' 1H',1,2.259e-314,3,b'F',b'0',48,2,322.8,b'H',1,24
1,2,b' 1H',1,2.265e-314,3,b'L',b'0',48,2,322.8,b'H',1,24
2,1,b' 2D',2,2.375e-314,3,b'L',b'0',52,2,76.81,b'D',1,24
3,2,b' 2D',2,2.265e-314,3,b'R',b'N',31,2,76.81,b'D',1,24
4,1,b' 3F',3,2.265e-314,1,b'F',b'0',42,2,259.4,b'F',1,24


In [12]:
cds['gv_15'].head()

,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CASEID,CASENO,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,RATWGT,PREEVENT,PREMOVE,PREILOC,PREISTAB,PSU,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,STRATIF,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHNO,VEHUSE,VIN,VINLNGTH,VERSION,DRUGS,VAIS,VAIS08,VINMODYR,MCYCLDS,VEHWGT,VINMAKE,VINAMOD,SERTR,VINBT,ROOF1,ROOF2,ROOF3,ANTILOCK,FRTWHLDR,FOURWHDR,RESTYPE,CARBUR,FUELCODE,WGTCDTR,VEHTYPE,WHLDRWHL,DAYRUNLT,otvehwgt,otbdytyp,VINJSER,VINJSER8,VINJURED,VINJURD8,VTREAT
0,1,50,nan,280,90,14,50,31,0,b' 1K',1,0,231,4,0,nan,0,1,1.241e+04,4,0,nan,nan,nan,nan,nan,0,nan,6,2,1,7,7,482,1997,1,1,36.63,5,14,2,3,2,3,3,4,2,nan,0,1,1,7,2,6,2,10,1,0,0,89,b'K',0,1,0,1,0,777,0,2,1,0,b'3B7KF23Z5VM5',17,28,0,nan,nan,1997,nan,nan,b'DODG',b'RPC',b'250',b'PC',1,nan,nan,4,NaN,NaN,b'M',NaN,b'G',2,b'T',44.01,b'N',nan,63,nan,nan,nan,nan,nan
1,nan,51,nan,nan,nan,14,nan,63,nan,b' 1K',1,nan,nan,nan,nan,nan,0,nan,1.241e+04,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2,1,86,nan,881,2013,nan,nan,36.63,62,1,1,1,2,1,3,4,2,nan,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,89,b'K',nan,0,0,1,0,777,0,nan,2,0,b'1M2AX02C0DM0',17,28,0,nan,nan,2013,nan,nan,b'MACK',b'700',b'GU7',b'CC',nan,nan,nan,nan,NaN,NaN,NaN,NaN,b'D',8,b'T',42,NaN,nan,nan,nan,nan,nan,nan,nan
2,nan,1,nan,998,998,18,nan,2,nan,b' 2D',2,1,110,nan,0,nan,1,1,1.249e+04,0,0,nan,nan,nan,nan,nan,0,nan,0,2,2,37,1,31,2001,1,1,69.79,13,14,4,1,2,2,2,1,2,nan,0,5.65e-313,0,0,0,1,8,888,0,0,0,72,b'D',0,1,0,1,0,777,0,1.043e-312,1,0,b'1HGEM22961L0',17,28,0,nan,nan,2001,nan,2590,b'HOND',b'SIX',NaN,b'CP',3,nan,nan,2,b'Y',b'N',b'E',b'F',b'G',nan,b'P',nan,b'N',nan,nan,nan,nan,nan,nan,nan
3,1,22,nan,350,350,18,25,4,0,b' 3E',3,0,154,0,0,1,0,1,1.24e+04,3,4,6.79e-313,21,21,404,2,0,998,3,4,1,63,1,34,2014,1,1,375.5,53,5,1,1,2,2,1,2,2,1,3,0,0,0,0,1,8,888,0,0,0,72,b'E',0,1,0,1,0,777,0,1.043e-312,1,0,b'5XXGR4A79EG3',17,28,0,1,1,2014,nan,3385,b'KIA',b'OSX',NaN,b'4D',1,3,nan,2,b'Y',b'N',b'7',b'F',b'G',nan,b'P',nan,b'S',143,4,0,0,1,1,4
4,1,20,nan,350,350,18,17,4,nan,b' 3E',3,0,143,0,0,nan,0,1,1.249e+04,3,4,4.753e-312,-22,22,164,nan,nan,998,0,4,1,51,1,42,1996,1,1,375.5,50,14,1,1,2,2,1,2,2,nan,3,0,0,0,0,1,8,888,0,0,0,72,b'E',0,1,0,1,0,777,0,1.043e-312,2,0,b'YV1LS5544T13',17,28,0,nan,nan,1996,nan,3128,b'VOLV',b'8/G',NaN,b'4D',1,3,nan,2,b'Y',b'N',b'C',b'F',b'G',nan,b'P',nan,b'S',154,4,nan,nan,nan,nan,nan


In [13]:
cds['gv_15'].describe()
len(cds['gv_15'])

,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CASENO,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,RATWGT,PREEVENT,PREMOVE,PREILOC,PREISTAB,PSU,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHNO,VEHUSE,VINLNGTH,VERSION,DRUGS,VAIS,VAIS08,VINMODYR,MCYCLDS,VEHWGT,ROOF1,ROOF2,ROOF3,ANTILOCK,WGTCDTR,WHLDRWHL,otvehwgt,otbdytyp,VINJSER,VINJSER8,VINJURED,VINJURD8,VTREAT
count,3497,4624,352,4397,4392,4619,2432,4804,2906,4814,4415,4414,3312,4426,1352,4396,4426,4514,4426,4426,2227,2227,2227,2227,1705,4002,2227,4597,4631,4622,4781,2469,4804,4771,4597,4426,4814,4624,4621,4618,3931,4814,4631,4626,4552,4631,1896,4631,4358,4359,4359,4359,4357,4356,4209,4359,4082,4227,4601,4406,4631,4631,4814,4629,3737,4628,4359,4814,4775,4563,4814,4451,1840,1840,4814,270,2766,4586,2660,314,4586,2287,2286,3295,3492,2799,2799,2799,2799,1924
mean,1.283,52.18,8.528,352,352.1,17.55,24.73,12.67,0.8118,64.6,0.1932,165.7,2.113,0.1984,8.41,0.2102,0.9977,5.004e+04,3.906,1.368,0.1949,-14.66,24.62,617,1.842,0.2161,992.9,3.027,3.17,1.668,31.63,2.492,219,2006,1.192,1.467,847.9,38.29,5.44,1.777,1.249,42.98,1.296,1.432,1.654,1.904,1.451,1.132,0.1294,0.245,0.332,3.758,1.278,7.524,850.6,0.404,0.04949,0.1815,66.46,0.004993,0.8087,0.5165,1.038,0.7431,452.5,0.7165,0.1718,1.569,0.0511,16.99,28,1.259,1.445,1.538,2438,9674,3738,1.725,3.615,8.43,2.978,2.602,50.18,254.4,21.26,0.09325,0.08253,0.9914,0.9929,2.34
std,0.7401,34.44,10.83,360.7,360.9,3.901,16.36,14.67,10.4,42.41,0.9134,39.1,3.128,0.3988,23.22,1.016,0.04748,3.035e+04,7.989,1.715,13.18,21.4,15.52,898.5,0.6914,1.062,68.95,2.379,1.443,1.002,19.92,4.574,239.9,6.506,0.9966,0.8832,2566,24.48,6.074,1.329,0.7319,30.18,0.6419,3.455,1.131,0.4174,1.072,1.023,0.5242,0.9365,3.744,14.16,1.082,1.652,176.8,3.797,0.4455,0.7438,21.25,0.07049,0.3934,0.9067,0.1944,0.9654,363.6,0.9723,0.7707,0.7348,0.4702,0.2852,0,2.671,1.85,2.244,1807,1663,2097,1.962,1.825,1.307,2.001,2.735,17.97,257.9,28.95,0.3406,0.3163,0.9098,0.9109,2.147
min,1,1,0,0,0,11,0,1,0,1,0,69,0,0,0,0,0,1,0,0,-84,-121,0,0,1,0,27,0,1,1,2,0,1,1954,0,0,2.685,1,0,0,0,2,1,1,1,1,1,0,0,0,0,0,1,1,2,0,0,0,0,0,0,0,1,0,0,0,0,1,0,10,28,0,0,0,1954,599,0,1,1,3,1,1,42,69,1,0,0,0,0,0
25%,1,20,0,90,90,18,13.75,4,0,28,0,139,0,0,1,0,1,1.951e+04,1,1.019e-312,-5,-25,14,138,2,0,998,0,2,1,13,1,32,2002,1,1,58.06,13,1,1,1,8.002,1,1,1,2,1,1.347e-312,4.346e-313,0,0,0,1,8,888,0,0,0,56,0,1,0,1,0,56,0,5.215e-313,1,0,17,28,0,0,0,2002,9999,2809,1,3,9,2,1,42,142,4,0,0,0,0,0
50%,1,65,0,210,210,18,20,4,0,58,0,157,0,0,1,0,1,4.632e+04,3,1,0,-14,20,298,2,0,998,3,3,1,30,1,47,2006,1,1,192.7,50,1,1,1,45,1,1,1,2,1,2,2.347e-312,0,0,0,1,8,888,0,0,0,64,0,1,0,1,0,777,3.651e-312,2.206e-312,1,0,17,28,0,1,1,2007,9999,3192,1,3,9,2,1,44,162,6,0,0,1,1,3
75%,1,86,19,350,350,18,31,15,0,98,0,186.8,6,0,2,0,1,8.023e+04,4,4,5,-6,31,705.5,2,0,998,6,4,3,49,2,421,2012,1,2,715.3,62,11,2,1,76,1,1,2,2,2,2,4.042e-312,0,0,0,1,8,888,0,0,0,80,0,1,1,1,2,777,1,4.259e-312,2,0,17,28,0,2,2,2013,9999,3550,1,3,9,3,2,44.01,206,20,0,0,1,1,4
max,9,98,40,998,998,98,127,97,454,160,8,334,8,1,98,8,1,1e+05,98,4.001,81,88,123,9154,8,7,998,7,7,5,98,98,998,2016,8,9,5.463e+04,98,97,7,7,82,3,88,5,8,8,3,8.002,8,98,88,7,8,998,98,7,8,121,1,1,8,3,2,777,3,8.001,6,7,17,28,8,7,9,9999,9999,9999,9.001,9.002,9.002,9,9.002,99.02,998,98.01,4,4,7,7,8


4814

In [14]:
cds['oi_11'].head()

,AIS,ASPECT90,CASEID,CASENO,DIRINJ,INJLEVEL,INJNO,INJSOU,INTRUNO,OCCNO,PSU,RATWGT,REGION90,SOUCON,SOUDAT,STRATIF,STRUSPEC,STRUTYPE,VEHNO,VERSION,BODYREG,LESION,SYSORG,REGION08,STRTYP08,STRSPC08,INJLVL08,AIS08
0,1,4,b' 5K',5,1,2,1,3,0,1,2,42.13,2,2,3,b'K',4,9,2,24,b'F',b'C',b'I',2,1,4,2,1
1,1,1,b' 6E',6,1,2,1,24,0,1,2,243.5,8,2,3,b'E',4,9,1,24,b'Y',b'C',b'I',8,1,4,2,1
2,1,4,b' 20F',20,1,2,1,152,0,1,2,64.5,4,1,3,b'F',4,9,2,24,b'C',b'C',b'I',4,1,4,2,1
3,1,1,b' 20F',20,1,2,2,24,0,1,2,64.5,8,2,3,b'F',4,9,2,24,b'Y',b'C',b'I',8,1,4,2,1
4,1,nan,b' 20F',20,7,2,3,697,0,1,2,64.5,1,nan,3,b'F',4,9,2,24,b'H',b'C',b'I',1,1,4,2,1


In [15]:
len(pd.merge(cds['oa_10'], cds['gv_10'], on = ['CASEID', 'CASENO', 'PSU', 'RATWGT', 'STRATIF', 'VERSION', 'VEHNO'], how = 'inner'))
print(len(cds['gv_10']), len(cds['oa_10']))

10215

8921 10215


In [16]:
cds['ve_11'].head()

,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,CASEID,CASENO,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELTYP1,FUELTYP2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,PSU,RATWGT,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,STRATIF,SVL1,SVL2,TDD1,TDD2,VEHNO,WHEELBAS,VERSION
0,1,2,0,154,b' 1H',1,85,0,6,6,6,5,5,6,16,158,9,9,158,1,1,nan,nan,nan,0,0,0,1,nan,b'F',b'L',nan,nan,nan,nan,1,nan,1,nan,nan,48,48,280,260,2,322.8,2,2,2,1,2,1,-117,131,b'Z',b'Z',b'H',b'E',b'E',b'W',b'W',1,267,24
1,1,2,0,148,b' 2D',2,35,0,0,17,33,11,2,1,146,130,9,0,998,3,2,nan,nan,nan,0,0,0,1,nan,b'L',b'R',nan,nan,nan,nan,1,nan,1,nan,nan,52,31,270,998,2,76.81,nan,nan,nan,nan,nan,nan,nan,nan,b'F',b'D',b'D',b'E',b'A',b'N',b'O',1,269,24
2,1,nan,0,146,b' 3F',3,28,0,0,2,14,30,5,0,6,136,12,nan,136,2,nan,nan,nan,nan,0,0,0,1,nan,b'F',NaN,nan,nan,nan,nan,1,nan,1,nan,nan,42,nan,0,nan,2,259.4,nan,nan,nan,nan,nan,nan,nan,nan,b'C',NaN,b'F',b'E',NaN,b'N',NaN,1,252,24
3,1,nan,0,158,b' 5K',5,250,0,0,0,8,11,8,3,-19,287,11,nan,998,1,nan,nan,nan,nan,0,0,0,1,nan,b'L',NaN,nan,nan,nan,nan,1,nan,1,nan,nan,2,nan,340,nan,2,42.13,nan,nan,nan,nan,nan,nan,nan,nan,b'D',NaN,b'K',b'E',NaN,b'W',NaN,1,287,24
4,1,3,0,152,b' 5K',5,25,1,41,27,23,15,6,2,-57,144,12,0,144,5,3,nan,nan,nan,0,0,0,1,nan,b'F',b'T',nan,nan,nan,nan,1,nan,1,nan,nan,1,31,350,998,2,42.13,nan,nan,nan,nan,nan,nan,nan,nan,b'L',b'Y',b'K',b'E',b'D',b'E',b'O',2,270,24


In [17]:
cds['vi_11'].head()

,CASEID,CASENO,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,PSU,STRATIF,VEHNO,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RATWGT,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT,VERSION
0,b' 1H',1,0,0,0,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,0,1,1,2,2,0,2,2,0,1,4,4,4,0,4,4,0,1,1,1,1,1,0,nan,2,b'H',1,0,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,140,322.8,0,0,0,2,0,24
1,b' 5K',5,0,0,0,0,0,1,1,1,0,1,1,1,2,1,1,1,0,1,1,1,1,2,2,2,0,2,2,2,1,3,2,3,0,2,3,3,1,1,3,1,1,1,nan,2,b'K',1,0,2,3,21,36,2,3,11,38,1,3,21,18,1,3,21,7,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,119,42.13,0,0,0,2,0,24
2,b' 5K',5,0,0,0,0,0,1,6,6,0,1,1,1,2,1,1,1,0,1,1,1,1,1,nan,2,0,2,2,2,1,11,nan,11,0,11,11,3,1,1,1,1,1,1,6,2,b'K',2,0,2,1,11,15,2,1,11,16,1,1,12,16,1,1,12,15,1,1,13,16,1,1,11,6,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,129,42.13,0,0,0,1,0,24
3,b' 6E',6,0,0,0,0,0,1,1,1,1,1,1,0,2,1,1,1,1,1,1,0,1,1,2,2,1,2,2,0,1,2,2,2,2,2,2,0,1,1,1,1,1,1,nan,2,b'E',1,0,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,57,243.5,0,0,0,1,0,24
4,b' 7H',7,0,0,0,0,0,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,2,2,0,2,2,2,1,3,2,3,0,2,3,3,1,1,1,1,1,1,nan,2,b'H',1,0,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,161,1132,nan,nan,0,nan,0,24


## 整数の値が入っているはずなのに小数点以下の値が入っているもの，同じ値なのにpython内部で別の値として認識されているものを修正

In [18]:
for col in ['PSU', 'VERSION', 'VEHNO']:
    for y in ['11', '14', '15']:
        for og in ['oa', 'gv', 've', 'vi']:
            cds['{}_{}'.format(og, y)][col] = cds['{}_{}'.format(og, y)][col].astype(np.int64)
        if col !=  'VEHNO':
            og = 'accident'
            cds['{}_{}'.format(og, y)][col] = cds['{}_{}'.format(og, y)][col].astype(np.int64)

"\nfor col in ['PSU', 'VERSION', 'VEHNO']:\n    for y in ['11', '14', '15']:\n        for og in ['oa', 'gv', 've', 'vi']:\n            cds['{}_{}'.format(og, y)][col] = cds['{}_{}'.format(og, y)][col].astype(np.int64)\n        if col !=  'VEHNO':\n            og = 'accident'\n            cds['{}_{}'.format(og, y)][col] = cds['{}_{}'.format(og, y)][col].astype(np.int64)\n"

In [19]:
len(pd.merge(cds['oa_10'], cds['gv_10'], on = ['CASEID', 'CASENO', 'PSU', 'RATWGT', 'STRATIF', 'VERSION', 'VEHNO'], how = 'inner'))
print(len(cds['gv_10']), len(cds['oa_10']))

10215

8921 10215


## テーブル，年毎のレコード数を表示

In [20]:
for year in range(10, 16):
    print(year)
    for file in file_name:
        print('{}:'.format(file) + str(len(cds['{}_{}'.format(file, year)])))

10
accident:4856
event:9065
gv:8921
oa:10215
oi:14276
ve:6618
vi:3475
11
accident:4278
event:8170
gv:7898
oa:8940
oi:11790
ve:5733
vi:2958
12
accident:3581
event:6755
gv:6665
oa:7402
oi:9063
ve:3509
vi:2003
13
accident:3385
event:6419
gv:6348
oa:7084
oi:8764
ve:3780
vi:2010
14
accident:2896
event:5522
gv:5445
oa:5955
oi:6975
ve:3857
vi:1901
15
accident:2633
event:4834
gv:4814
oa:5479
oi:5946
ve:3273
vi:1592


## 年毎に同一のカラムを表示

In [21]:
for year in range(10, 16):
    print(year)
    same_column(cds, str(year)).head(7)

10


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,OCCNO
accident,1,1,1,1,1,1,nan,nan
event,1,1,1,1,1,1,nan,nan
gv,1,1,1,1,1,1,1,nan
oa,1,1,1,1,1,1,1,1
oi,1,1,1,1,1,1,1,1
ve,1,1,1,1,1,1,1,nan
vi,1,1,1,1,1,1,1,nan


11


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,OCCNO
accident,1,1,1,1,1,1,nan,nan
event,1,1,1,1,1,1,nan,nan
gv,1,1,1,1,1,1,1,nan
oa,1,1,1,1,1,1,1,1
oi,1,1,1,1,1,1,1,1
ve,1,1,1,1,1,1,1,nan
vi,1,1,1,1,1,1,1,nan


12


...

,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,OCCNO
accident,1,1,1,1,1,1,nan,nan
event,1,1,1,1,1,1,nan,nan
gv,1,1,1,1,1,1,1,nan
oa,1,1,1,1,1,1,1,1
oi,1,1,1,1,1,1,1,1
ve,1,1,1,1,1,1,1,nan
vi,1,1,1,1,1,1,1,nan


14


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,OCCNO
accident,1,1,1,1,1,1,nan,nan
event,1,1,1,1,1,1,nan,nan
gv,1,1,1,1,1,1,1,nan
oa,1,1,1,1,1,1,1,1
oi,1,1,1,1,1,1,1,1
ve,1,1,1,1,1,1,1,nan
vi,1,1,1,1,1,1,1,nan


15


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,OCCNO
accident,1,1,1,1,1,1,nan,nan
event,1,1,1,1,1,1,nan,nan
gv,1,1,1,1,1,1,1,nan
oa,1,1,1,1,1,1,1,1
oi,1,1,1,1,1,1,1,1
ve,1,1,1,1,1,1,1,nan
vi,1,1,1,1,1,1,1,nan


## テーブル毎の結合keyを宣言

In [22]:
merge_key = {}
for file in file_name:
    if file == 'accident' or file == 'event':
        merge_key[file] =  ['CASEID', 'CASENO', 'PSU', 'RATWGT', 'STRATIF', 'VERSION']
    if file == 'gv' or file == 've' or file == 'vi':
        merge_key[file] = ['CASEID', 'CASENO', 'PSU', 'RATWGT', 'STRATIF', 'VERSION', 'VEHNO']
    if file == 'oa' or file == 'oi':
        merge_key[file] = ['CASEID', 'CASENO', 'PSU', 'RATWGT', 'STRATIF', 'VERSION', 'VEHNO', 'OCCNO']

## 同一の結合keyに対して重複しているレコードを表示

In [23]:
dup_dic = {}
for file in file_name:
    print(file)
    dup_dic[file] = duplicate(cds, '{}_10'.format(file), merge_key[file])
    dup_dic[file].head()

accident


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,AAIS,AAIS08,AINJSER,AINJSER8,AINJURED,AINJURD8,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS


event


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,ACCSEQ,CLASS2,CLASS1,GADEV1,GADEV2,OBJCONT,VEHNUM
304,b'145G',145,2,1319,b'G',23,7,5.432e-312,3,b'F',b'0',50,1
305,b'145G',145,2,1319,b'G',23,8.001,5.411e-312,3,b'F',b'0',42,1
1024,b' 74F',74.02,5,128.5,b'F',23,1,20,14,b'F',b'F',2,1
1025,b' 74F',74.02,5,128.5,b'F',23,2,3,20,b'B',b'F',3,2
1534,b' 89K',89.02,8,65.22,b'K',23,1,1.952e-312,14,b'F',b'0',43,1


gv


...

,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,OCCNO,AIS,ASPECT90,DIRINJ,INJLEVEL,INJNO,INJSOU,INTRUNO,REGION90,SOUCON,SOUDAT,STRUSPEC,STRUTYPE,BODYREG,LESION,SYSORG,REGION08,STRTYP08,STRSPC08,INJLVL08,AIS08
0,b' 1C',1,2,41.05,b'C',23,1,1,2,2,2,4,1,251,0,8,2,3,34,5,b'Q',b'F',b'S',8,5,43,31,2
1,b' 1C',1,2,41.05,b'C',23,1,1,nan,2,1,nan,2,4,0,nan,2,3,nan,nan,NaN,NaN,NaN,7,4,0,99,9
2,b' 5K',5,2,43.51,b'K',23,1,1,1,2,1,2,1,152,0,7,1,7,4,9,b'X',b'C',b'I',7,1,4,2,1
3,b' 5K',5,2,43.51,b'K',23,1,1,1,4,1,2,2,152,0,4,1,7,4,9,b'C',b'C',b'I',4,1,4,2,1
4,b' 5K',5,2,43.51,b'K',23,1,1,1,3,1,2,3,152,0,8,1,7,4,9,b'Y',b'C',b'I',8,1,4,2,1


ve


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELTYP1,FUELTYP2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,SVL1,SVL2,TDD1,TDD2,TOWRES,WHEELBAS


vi


,CASEID,CASENO,PSU,RATWGT,STRATIF,VERSION,VEHNO,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT


## テーブルの結合
oi,eventテーブルを除き,2010~2015年のデータをそれぞれ結合する

In [24]:
cds_merge = {}
for year in range(10, 16):
    year = str(year)
    cds_merge[year] = cds['oa_{}'.format(year)]
    for file in [x for x in file_name if not (x == 'oa' or x == 'oi' or x == 'event')]:
        if file != 'gv':
            cds_merge[year] = pd.merge(cds_merge[year], cds['{}_{}'.format(file, year)], on = merge_key[file], how = 'left')
        else:
            cds_merge[year] = pd.merge(cds_merge[year], cds['{}_{}'.format(file, year)], on = merge_key[file], how = 'inner')
    print(year)
    print('oa_length:' + str(len(cds['oa_{}'.format(year)])))
    print('merge_length:' + str(len(cds_merge[year])))
    cds_merge[year].head(10)

10
oa_length:10215
merge_length:10215


,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CASEID,CASENO,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJNUM,INJNUM08,INJSEV,ISS,BAGMAINT,MAIS,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,OCCNO,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,RATWGT,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,STRATIF,SEATRACK,TREATMNT,VEHNO,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,ISS08,MAIS08,INTGREST,VERSION,STORIENT,GLASGOW,BLOOD,BICARB,AAIS,AAIS08,AINJSER,AINJSER8,AINJURED,AINJURD8,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,PREEVENT,PREMOVE,PREILOC,PREISTAB,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHUSE,VIN,VINLNGTH,DRUGS,VAIS,VAIS08,VINMODYR,MCYCLDS,VEHWGT,VINMAKE,VINAMOD,SERTR,VINBT,ROOF1,ROOF2,ROOF3,ANTILOCK,FRTWHLDR,FOURWHDR,RESTYPE,CARBUR,FUELCODE,WGTCDTR,VEHTYPE,WHLDRWHL,DAYRUNLT,otvehwgt,otbdytyp,VINJSER,VINJSER8,VINJURED,VINJURD8,VTREAT,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELTYP1,FUELTYP2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,SVL1,SVL2,TDD1,TDD2,TOWRES,WHEELBAS,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT
0,23,1,7,1,1,1,1,8,2,b' 1C',1,0,0,0,2,0,0,0,0,0,0,0,-15,0,0,0,0,1,2,3,157,0,1,2,3,4,nan,2,4,1,4,0,2,2,4,1,1,7,4,8,1,nan,nan,2,41.05,1,1,11,1,2,1,23,b'C',3,4,1,66,nan,1,nan,0,0,4,2,0,23,1,15,1,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2,7,18,997,997,14,nan,4,0,0,125,7,0,1,1,1,1.256e+04,5,0,nan,nan,nan,nan,nan,0,nan,3,2,2,55,9,35,2007,2,2,6,14,4,2,2,3,1,2,1,0,1,4,2,44,3,1,20,2,0,0,56,0,1,0,1,0,777,0,4,0,b'KMHDU46D37U1',17,0,2,2,2007,nan,2723,b'HYUN',b'GST',NaN,b'4D',1,3,nan,2,b'Y',b'N',b'7',b'F',b'G',nan,b'P',nan,b'N',998,98,0,0,1,1,4,2,1,0,154,120,1,nan,nan,nan,nan,nan,nan,nan,nan,0,12,998,3,1,nan,nan,nan,0,1.358e-312,0,1,nan,b'T',b'F',nan,nan,nan,nan,1,nan,1,nan,nan,31,44,998,350,8,8,3,0,0,0,-55.01,158,b'Y',b'L',b'D',b'E',b'O',b'W',1,265,0,0,0,0,0,6,1,6,0,1,1,0,2,1,1,1,0,1,1,0,1,1,2,2,0,3,2,0,1,2,2,2,0,2,2,0,1,1,1,1,3,0,11,0,2,1,12,13,2,1,11,15,2,1,12,15,2,1,11,13,2,1,13,13,2,1,21,13,1,1,13,15,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,80,0,0,0,1,4.172e-312
1,31,1,7,1,1,1,1,8,4,b' 1C',1,0,0,0,2,0,2.716e-312,0,0,0,0,0,-15,0,0,0,0,1,2,3,168,0,0,2.825e-312,0,0,nan,0,4,0,0,0,0,1,4,2,1,7,4,8,3,nan,nan,2,41.05,2,1,13,1,1,1,23,b'C',3,0,1,82,nan,nan,nan,0,0,0,0,0,23,1,0,1,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2,7,18,997,997,14,nan,4,0,0,125,7,0,1,1,1,1.256e+04,5,0,nan,nan,nan,

11
oa_length:8940
merge_length:1913


,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CASEID,CASENO,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJNUM,INJNUM08,INJSEV,ISS,BAGMAINT,MAIS,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,OCCNO,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,RATWGT,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,STRATIF,SEATRACK,TREATMNT,VEHNO,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,ISS08,MAIS08,INTGREST,CHOWUSED,VERSION,STORIENT,GLASGOW,BLOOD,BICARB,AAIS,AAIS08,AINJSER,AINJSER8,AINJURED,AINJURD8,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,PREEVENT,PREMOVE,PREILOC,PREISTAB,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHUSE,VIN,VINLNGTH,DRUGS,VAIS,VAIS08,VINMODYR,MCYCLDS,VEHWGT,VINMAKE,VINAMOD,SERTR,VINBT,ROOF1,ROOF2,ROOF3,ANTILOCK,FRTWHLDR,FOURWHDR,RESTYPE,CARBUR,FUELCODE,WGTCDTR,VEHTYPE,WHLDRWHL,DAYRUNLT,otvehwgt,otbdytyp,VINJSER,VINJSER8,VINJURED,VINJURD8,VTREAT,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELTYP1,FUELTYP2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,SVL1,SVL2,TDD1,TDD2,WHEELBAS,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT
0,25,1,2,3,96,96,nan,8,5,b' 69D',69,0,0,0,6,0,0,0,0,0,0,0,996,0,0,0,0,3,3,nan,178,0,1,1,1,1,nan,1,4,nan,4,0,1,3,3,1,1,7,8,8,3,nan,nan,3,21.15,1,nan,11,nan,1,96,nan,b'D',nan,4,1,93,nan,2,2,0,0,1,1,nan,0,24,nan,15,1,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,69,nan,320,110,18,nan,4,nan,0,144,nan,0,nan,0,1,6106,0,0,nan,nan,nan,nan,2,0,nan,0,4,4,37,1,32,2006,3,3,62,1,1,1,1,1,1,2,2,2,0,0,0,0,1,8,888,0,0,0,48,0,1,1,1,2,777,0,0,0,b'1HGCM56326A0',17,0,1,1,2006,nan,3197,b'HOND',b'ASE',NaN,b'4D',1,nan,nan,2,b'Y',b'N',b'S',b'F',b'G',nan,b'P',nan,b'S',nan,nan,0,0,3,3,4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,27,1,2,3,96,96,nan,8,5,b' 69D',69,0,0,0,6,0,0,0,0,0,0,0,996,0,0,0,0,3,3,nan,175,0,97,97,3,0,nan,7,4,nan,4,0,2,1,3,2,1,7,8,8,3,nan,nan,3,21.15,2,nan,13,nan,1,96,nan,b'D',

12
oa_length:7402
merge_length:7402


...

,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CASEID,CASENO,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJNUM,INJNUM08,INJSEV,ISS,BAGMAINT,MAIS,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,OCCNO,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,RATWGT,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,STRATIF,SEATRACK,TREATMNT,VEHNO,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,ISS08,MAIS08,INTGREST,CHOWUSED,LATCHDES,LATCHUSE,POSPRES,POSUSE,POSGUIDE,VERSION,STORIENT,GLASGOW,BLOOD,BICARB,AAIS,AAIS08,AINJSER,AINJSER8,AINJURED,AINJURD8,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,PREEVENT,PREMOVE,PREILOC,PREISTAB,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHUSE,VIN,VINLNGTH,DRUGS,VAIS,VAIS08,VINMODYR,MCYCLDS,VEHWGT,VINMAKE,VINAMOD,SERTR,VINBT,ROOF1,ROOF2,ROOF3,ANTILOCK,FRTWHLDR,FOURWHDR,RESTYPE,CARBUR,FUELCODE,WGTCDTR,VEHTYPE,WHLDRWHL,DAYRUNLT,otvehwgt,otbdytyp,VINJSER,VINJSER8,VINJURED,VINJURD8,VTREAT,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELTYP1,FUELTYP2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,SVL1,SVL2,TDD1,TDD2,WHEELBAS,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT
0,17,1,7,1,1,1,1,1,2,b' 1H',1,0,0,0,1,0,0,1.358e-312,0,0,0,0,-41,0,0,0,0,1,2,3,175,0,0,0,2.651e-312,0,1,0,4,1,4,0,0,1,4,1,1,7,4,8,1,0,1,2,282.8,1,1,11,1,1,1,23,b'H',6,0,1,73,97,1,2,0,0,0,0,0,0,0,0,0,3,0,26,1,0,1,0,0,0,0,0,0,0,2,0,6,2,1,0,1,3,825,1,2013,0,1,2,nan,998,998,19,40,20,0,1,189,0,1,5,0,1,1.253e+04,1,1,0,-41,41,1204,2,0,998,3,2,1,35,9,443,2007,1,1,5,14,4,2,3,5,4,2,1,0,0,0,0,0,1,8,888,0,0,0,89,0,1,0,1,0,nan,0,0,0,b'5N1BV28U57N1',17,0,0,0,2007,nan,nan,b'NISS',b'QST',b'Q35',b'SV',1,3,nan,2,NaN,NaN,b'S',NaN,b'G',1,b'T',42,b'U',998,98,0,0,0,0,0,1,nan,0,170,61,0,0,28,43,46,11,0,-7,171,12,nan,171,3,nan,nan,nan,nan,0,0,0,1,nan,b'F',NaN,nan,nan,nan,nan,1,nan,1,nan,nan,42,nan,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,b'Y',NaN,b'E',NaN,b'W',NaN,315,0,0,0,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,0,1,1,2,1,0,2,1,0,1,3,2,3,0,2,3,0,1,1,1,1,1,1,nan,0,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,138,0,0,0,5,0
1,72,nan,7,nan,nan,nan,nan,nan,nan,b' 2F',2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1,nan,nan,nan,nan,nan,nan,nan,nan,0,nan,1,nan,nan,4,8,nan,nan,nan,2,238.2,1,nan,11,nan,2,nan,nan,b'F',nan,nan,1,nan,nan,nan,nan,2.716e-312,nan,nan,nan,nan,nan,nan,nan

14
oa_length:5955
merge_length:40


,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CASEID,CASENO,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJNUM,INJNUM08,INJSEV,ISS,BAGMAINT,MAIS,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,OCCNO,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,RATWGT,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,STRATIF,SEATRACK,TREATMNT,VEHNO,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,ISS08,MAIS08,INTGREST,CHOWUSED,LATCHDES,LATCHUSE,POSPRES,POSUSE,POSGUIDE,VERSION,STORIENT,GLASGOW,BLOOD,BICARB,AAIS,AAIS08,AINJSER,AINJSER8,AINJURED,AINJURD8,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,PREEVENT,PREMOVE,PREILOC,PREISTAB,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHUSE,VIN,VINLNGTH,DRUGS,VAIS,VAIS08,VINMODYR,MCYCLDS,VEHWGT,VINMAKE,VINAMOD,SERTR,VINBT,ROOF1,ROOF2,ROOF3,ANTILOCK,FRTWHLDR,FOURWHDR,RESTYPE,CARBUR,FUELCODE,WGTCDTR,VEHTYPE,WHLDRWHL,DAYRUNLT,otvehwgt,otbdytyp,VINJSER,VINJSER8,VINJURED,VINJURD8,VTREAT,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELTYP1,FUELTYP2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,SVL1,SVL2,TDD1,TDD2,WHEELBAS,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT
0,53,1,2,1,1,1,1,8,2,b' 99F',99,0,0,0,1,0,0,1.698e-312,0,4.074e-312,0,0,-12,0,0,0,0,1,2,3,nan,0,0,0,1,0,nan,0,4,1,4,0,0,4,nan,1,1,7,3,8,1,nan,nan,12,166.8,1,1,11,1,2,1,23,b'F',5,0,2,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,3,0,27,1,0,1,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1,66,nan,270,60,19,12,4,0,0,165,0,1,nan,0,1,4.843e+04,1,4,0,-12,12,108,nan,0,998,3,2,1,24,1,9,2007,1,1,62,1,1,1,1,1,1,2,nan,0,0,0,0,0,1,8,888,0,0,0,72,0,1,0,1,0,nan,0,0,0,b'1G8ZV57737F2',17,0,4.563e-312,5.215e-312,2007,nan,3647,b'STRN',b'AXR',NaN,b'4D',1,3,6,2,b'Y',b'N',b'S',b'F',b'G',nan,b'P',nan,b'S',152,4,0,0,0,0,0,1,nan,0,153,31,0,0,0,0,0,0,2,58,148,12,nan,148,4,nan,nan,nan,nan,0,0,0,1,nan,b'F',NaN,nan,nan,nan,nan,1,nan,1,nan,nan,1,nan,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,b'R',NaN,b'E',NaN,b'E',NaN,285,0,0,0,0,0,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,2,2,0,2,2,2,1,2,11,2,0,11,2,3,1,1,1,1,1,0,nan,0,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,123,0,0,0,1,0
1,33,1,3,7,97,97,1,1,2,b'100H',100,0,0,0,7,0,0,1.358e-312,0,4.074e-312,0,0,997,0,0,0,0,7,7,3,180,0,0,0,1,0,1,0,4,1,4,0,0,7,4,1,1,7,4,8,1,0,1,12,624.6,1,1,11,1,1,97,23,b'H',3,0,1,78,97,1,2,0,0,2.716e-312,0,0,0,0,0,0,3,0,27,1,0,1,0,nan,na

15
oa_length:5479
merge_length:805


,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CASEID,CASENO,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJNUM,INJNUM08,INJSEV,ISS,BAGMAINT,MAIS,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,OCCNO,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,RATWGT,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,STRATIF,SEATRACK,TREATMNT,VEHNO,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,ISS08,MAIS08,INTGREST,CHOWUSED,LATCHDES,LATCHUSE,POSPRES,POSUSE,POSGUIDE,VERSION,STORIENT,GLASGOW,BLOOD,BICARB,AAIS,AAIS08,AINJSER,AINJSER8,AINJURED,AINJURD8,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,PREEVENT,PREMOVE,PREILOC,PREISTAB,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHUSE,VIN,VINLNGTH,DRUGS,VAIS,VAIS08,VINMODYR,MCYCLDS,VEHWGT,VINMAKE,VINAMOD,SERTR,VINBT,ROOF1,ROOF2,ROOF3,ANTILOCK,FRTWHLDR,FOURWHDR,RESTYPE,CARBUR,FUELCODE,WGTCDTR,VEHTYPE,WHLDRWHL,DAYRUNLT,otvehwgt,otbdytyp,VINJSER,VINJSER8,VINJURED,VINJURD8,VTREAT,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELTYP1,FUELTYP2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,SVL1,SVL2,TDD1,TDD2,WHEELBAS,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT
0,16,1,2,7,97,97,1,1,4,b' 12G',12,0,0,0,7,0,0,0,0,0,0,4.074e-312,997,0,0,0,0,7,7,3,185,0,0,5.389e-312,0,0,1,1.749e-312,4,1,4,0,0,7.001,4,1,1,1,4,8,1,0,1,2,368.7,1,1,11,1,1,97,23,b'G',5,0,1,79,97,7,2,0,2.347e-312,0,0,4.129e-312,0,0,0,0,3,0,28,1,0,1,0,0,0,0,0,0,0,2,0,7,2,2,0,3,3,2333,1,2015,0,1,2,nan,998,998,19,13,14,0,0,141,0,1,1,0,1,1.253e+04,1,4,4.074e-312,-13,13,107,2,0,998,3,2,3,2,6,407,2014,2,2,5,14,4,4,3,1,4,2,7,0,1,4,5,31,3,2,3,1,0,0,48,0,1,0,1,0,777,0,6.001,0,b'1C4NJPBAXED6',17,0,0,0,2014,nan,nan,b'JEEP',b'PRT',b'SPO',b'4W',1,nan,nan,2,NaN,NaN,b'W',NaN,b'G',1,b'T',42.01,b'O',998,98,0,0,0,0,0,1,2,0,151,143,0,0,0,0,0,1,4,1,148,12,0,148,1,1,nan,nan,nan,0,0,0,1,nan,b'F',b'R',nan,nan,nan,nan,17,nan,1,nan,nan,44,31,0,998,nan,nan,nan,nan,nan,nan,nan,nan,b'D',b'D',b'E',b'E',b'W',b'O',264,0,0,0,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,0,1,1,2,2,0,2,1,0,1,3,2,3,0,2,3,0,1,1,1,1,1,1,nan,0,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,3,0,0,0,3,0
1,17,1,2,7,97,97,1,1,4,b' 12G',12,0,0,0,7,0,0,0,2.716e-312,0,0,0,997,0,0,0,0,7,7,3,165,0,0,4.107e-312,0,0,1,2.608e-312,4,1,4,0,0,7.001,4,2,1,1,4,8,1,0,1,2,368.7,2,1,13,1,1,97,23,b'G',3,0,1,59,97,7,2,0,2.347e-312,0,0,4.129e-312,0,0,0,0,3,0,28,1,0,1,0,0,0,0,0,

In [25]:
cds_merge['10'].head()

,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CASEID,CASENO,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJNUM,INJNUM08,INJSEV,ISS,BAGMAINT,MAIS,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,OCCNO,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,RATWGT,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,STRATIF,SEATRACK,TREATMNT,VEHNO,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,ISS08,MAIS08,INTGREST,VERSION,STORIENT,GLASGOW,BLOOD,BICARB,AAIS,AAIS08,AINJSER,AINJSER8,AINJURED,AINJURD8,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,PREEVENT,PREMOVE,PREILOC,PREISTAB,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHUSE,VIN,VINLNGTH,DRUGS,VAIS,VAIS08,VINMODYR,MCYCLDS,VEHWGT,VINMAKE,VINAMOD,SERTR,VINBT,ROOF1,ROOF2,ROOF3,ANTILOCK,FRTWHLDR,FOURWHDR,RESTYPE,CARBUR,FUELCODE,WGTCDTR,VEHTYPE,WHLDRWHL,DAYRUNLT,otvehwgt,otbdytyp,VINJSER,VINJSER8,VINJURED,VINJURD8,VTREAT,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELTYP1,FUELTYP2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,SVL1,SVL2,TDD1,TDD2,TOWRES,WHEELBAS,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT
0,23,1,7,1,1,1,1,8,2,b' 1C',1,0,0,0,2,0,0,0,0,0,0,0,-15,0,0,0,0,1,2,3,157,0,1,2,3,4,nan,2,4,1,4,0,2,2,4,1,1,7,4,8,1,nan,nan,2,41.05,1,1,11,1,2,1,23,b'C',3,4,1,66,nan,1,nan,0,0,4,2,0,23,1,15,1,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2,7,18,997,997,14,nan,4,0,0,125,7,0,1,1,1,1.256e+04,5,0,nan,nan,nan,nan,nan,0,nan,3,2,2,55,9,35,2007,2,2,6,14,4,2,2,3,1,2,1,0,1,4,2,44,3,1,20,2,0,0,56,0,1,0,1,0,777,0,4,0,b'KMHDU46D37U1',17,0,2,2,2007,nan,2723,b'HYUN',b'GST',NaN,b'4D',1,3,nan,2,b'Y',b'N',b'7',b'F',b'G',nan,b'P',nan,b'N',998,98,0,0,1,1,4,2,1,0,154,120,1,nan,nan,nan,nan,nan,nan,nan,nan,0,12,998,3,1,nan,nan,nan,0,1.358e-312,0,1,nan,b'T',b'F',nan,nan,nan,nan,1,nan,1,nan,nan,31,44,998,350,8,8,3,0,0,0,-55.01,158,b'Y',b'L',b'D',b'E',b'O',b'W',1,265,0,0,0,0,0,6,1,6,0,1,1,0,2,1,1,1,0,1,1,0,1,1,2,2,0,3,2,0,1,2,2,2,0,2,2,0,1,1,1,1,3,0,11,0,2,1,12,13,2,1,11,15,2,1,12,15,2,1,11,13,2,1,13,13,2,1,21,13,1,1,13,15,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,80,0,0,0,1,4.172e-312
1,31,1,7,1,1,1,1,8,4,b' 1C',1,0,0,0,2,0,2.716e-312,0,0,0,0,0,-15,0,0,0,0,1,2,3,168,0,0,2.825e-312,0,0,nan,0,4,0,0,0,0,1,4,2,1,7,4,8,3,nan,nan,2,41.05,2,1,13,1,1,1,23,b'C',3,0,1,82,nan,nan,nan,0,0,0,0,0,23,1,0,1,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2,7,18,997,997,14,nan,4,0,0,125,7,0,1,1,1,1.256e+04,5,0,nan,nan,nan,

## 特徴量のドロップ
結合用の特徴量・AISを元にした特徴量・年によってデータが存在しない特徴量をドロップ

In [26]:
cds_prepro = {}
drop_columns = ['CASEID', 'CASENO', 'VEHNO', 'OCCNO', 'RATWGT', 'STRATIF', 'VERSION', 'AAIS',\
                'AAIS08', 'VAIS', 'VAIS08', 'MAIS', 'MAIS08', 'INJNUM08', 'ISS08', 'TOWRES', \
                'VIN', 'AINJSER', 'AINJSER8', 'AINJURED', 'AINJURD8', 'VINJSER', 'VINJSER8', \
                'VINJURED', 'VINJURD8', 'INJNUM', 'LATCHDES', 'LATCHUSE', 'POSPRES', 'POSUSE', \
                'POSGUIDE','CHOWUSED', 'FUELTYP1', 'FUELTYP2']#INJSEVは警察の報告でありAISデータを使用していない
for year in range(10, 16):
    year = str(year)
    cds_prepro[year] = cds_merge[year]
    year_columns = cds_prepro[year].columns.values.tolist()
    for column in drop_columns:
        if column in year_columns:
            cds_prepro[year] = cds_prepro[year].drop(columns = [column])

## 値の変更
同一のものを指しているにも関わらず年によって値が違うものを修正

In [27]:
for year in range(10, 16):
    year = str(year)
    cds_prepro[year]['CLIMATE'] = cds_prepro[year]['CLIMATE'].replace('13', '20') 

## ISSデータが存在しないレコードのドロップ
推定対象のISSが存在しないレコードのドロップ

In [28]:
#ISSのnanをdropし，年毎のレコード数をprint
for year in range(10, 16):
    year = str(year)
    cds_prepro[year] = cds_prepro[year].dropna(subset = ['ISS'])
    print(str(year) + ':' + str(len(cds_prepro[year])))

10:6229
11:1078
12:4094
13:3819
14:19
15:378


## 論文と年以外の条件を同じに
Vehicle Modelの0~490が相当

In [29]:
q_column = ['AGE', 'EJECTION', 'MODEL', 'MODELYR']
query = ['AGE >= 15', 'EJECTION == 0', '0 <= MODEL <= 490', '2001 <= MODELYR <= 2015']
for year in range(10, 16):
    year =  str(year)
    print(pycolor.GREEN + 'year:' + year + pycolor.END)
    print('レコード数:', len(cds_prepro[year]))
    for col, que in zip(q_column, query):
        print(pycolor.RED +'column:' + col + pycolor.END)
        print('NaN:', cds_prepro[year][col].isnull().sum())
        print('クエリによるレコードの減少数(' + que + '):', len(cds_prepro[year]) - len(cds_prepro[year].query(que)))
    print()

year:10
レコード数: 6229
column:AGE
NaN: 107
クエリによるレコードの減少数(AGE >= 15): 721
column:EJECTION
NaN: 21
クエリによるレコードの減少数(EJECTION == 0): 152
column:MODEL
NaN: 0
クエリによるレコードの減少数(0 <= MODEL <= 490): 4
column:MODELYR
NaN: 3
クエリによるレコードの減少数(2001 <= MODELYR <= 2015): 3

year:11
レコード数: 1078
column:AGE
NaN: 10
クエリによるレコードの減少数(AGE >= 15): 121
column:EJECTION
NaN: 5
クエリによるレコードの減少数(EJECTION == 0): 33
column:MODEL
NaN: 0
クエリによるレコードの減少数(0 <= MODEL <= 490): 6
column:MODELYR
NaN: 0
クエリによるレコードの減少数(2001 <= MODELYR <= 2015): 0

year:12
レコード数: 4094
column:AGE
NaN: 67
クエリによるレコードの減少数(AGE >= 15): 420
column:EJECTION
NaN: 36
クエリによるレコードの減少数(EJECTION == 0): 89
column:MODEL
NaN: 0
クエリによるレコードの減少数(0 <= MODEL <= 490): 20
column:MODELYR
NaN: 5
クエリによるレコードの減少数(2001 <= MODELYR <= 2015): 5

year:13
レコード数: 3819
column:AGE
NaN: 97
クエリによるレコードの減少数(AGE >= 15): 383
column:EJECTION
NaN: 48
クエリによるレコードの減少数(EJECTION == 0): 101
column:MODEL
NaN: 0
クエリによるレコードの減少数(0 <= MODEL <= 490): 4
column:MODELYR
NaN: 0
クエリによるレコードの減少数(2001 <= MODELYR <= 201

In [30]:
query = ['AGE >= 15', 'EJECTION == 0']#, '0 <= MODEL <= 490', '2001 <= MODELYR <= 2015']
for que in query:
    print('query:', que)
    print('クエリによる減少数(残ったレコード数)')
    for year in range(10, 16):
        year = str(year)
        before = len(cds_prepro[year])
        cds_prepro[year] = cds_prepro[year].query(que)
        print(' ', year, ':', str(before - len(cds_prepro[year])), '({})'.format(len(cds_prepro[year])))
        #print(' ', year, ':', str(before - len(cds_prepro[year].query(que))))

query: AGE >= 15
クエリによる減少数(残ったレコード数)
  10 : 721 (5508)
  11 : 121 (957)
  12 : 420 (3674)
  13 : 383 (3436)
  14 : 0 (19)
  15 : 31 (347)
query: EJECTION == 0
クエリによる減少数(残ったレコード数)
  10 : 141 (5367)
  11 : 33 (924)
  12 : 85 (3589)
  13 : 87 (3349)
  14 : 1 (18)
  15 : 10 (337)


In [31]:
cds_prepro['15']['VEHWGT'].isnull().sum()

89

## カラム名をCSVに出力

In [32]:
col = cds_prepro['15'].query('MODEL > 500')
path = os.path.dirname(os.path.abspath('__file__'))
col.to_csv(os.path.join(path, 'column.csv'))

## 15年の要約統計量

In [33]:
cds_prepro['15'].describe()

,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJSEV,ISS,BAGMAINT,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,SEATRACK,TREATMNT,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,INTGREST,STORIENT,GLASGOW,BLOOD,BICARB,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,PREEVENT,PREMOVE,PREILOC,PREISTAB,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHUSE,VINLNGTH,DRUGS,VINMODYR,MCYCLDS,VEHWGT,ROOF1,ROOF2,ROOF3,ANTILOCK,WGTCDTR,WHLDRWHL,otvehwgt,otbdytyp,VTREAT,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,WHEELBAS,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT
count,337,285,334,285,285,285,198,285,239,334,337,337,285,337,337,337,337,337,337,334,285,337,337,337,257,285,285,203,247,313,334,337,168,285,182,269,337,328,285,195,285,285,322,337,269,189,173,337,337,185,337,205,337,285,188,183,334,248,187,219,207,337,337,202,205,334,305,334,211,332,337,35,337,328,337,337,337,337,337,337,248,337,56,336,336,336,188,337,219,335,306,237,337,160,317,337,335,337,337,176,176,176,176,213,278,176,337,337,336,337,229,337,337,337,337,337,337,337,292,337,335,304,337,226,337,334,334,334,334,334,334,328,334,308,275,335,337,337,337,337,337,308,337,334,337,281,336,337,56,248,332,220,64,332,142,142,222,248,333,212,86,212,209,163,212,147,147,147,147,147,147,147,147,201,78,195,180,66,4,0,4,199,212,212,203,11,1,0,0,0,203,11,0,212,85,201,78,45,45,45,45,43,43,45,45,210,212,212,212,212,212,199,197,201,211,199,199,208,200,203,193,200,211,199,200,207,188,211,203,209,212,201,204,209,212,194,195,199,210,194,198,209,200,199,204,199,202,206,44,204,174,43,43,43,43,30,30,30,30,20,20,20,20,17,17,17,17,7,7,7,7,5,5,5,5,4,4,4,4,2,2,2,2,1,1,1,1,0,0,0,0,115,183,183,171,167,212
mean,37.99,0.9158,2.452,4.06,58.21,63.17,0.9394,3.789,2.774,0.002994,0.005935,0.01187,4.495,4.171e-313,8.463e-314,9.631e-313,6.105e-313,0,1.333e-312,0.002994,668,0,0,0,0.003891,4.404,4.67,2.936,170.3,0.754,1.051,2.674,0.8632,3.996,0.9066,3.758,0,1.064,4.628,3.615,0.8175,3.947,4.068,7.929,1.665,0.1905,1.139,77.68,1.255,1.751,12.15,1.527,1.475,63.16,20.24,4.301,2.359,73.46,30.79,1.676,1.749,2.762e-312,1.195e-313,0.004952,0.9951,13.44,1.007,11.1,1.749,2.687,4.015,1.543,2.226,2.54,6.736,10.14,1299,2.344,2015,0,1.246,53.68,4.214,273.4,282.9,17.17,22.97,7.997,0.1918,0.1731,161.6,1.574,0.3056,8.037,0.31

In [34]:
cds_prepro['15'].head(50)

,AGE,BAGAVAIL,BAGAVRPT,BAGDEPLY,BAGEVENT,BAGDAMAG,BAGFAIL,BAGTYPE,BELTANCH,CAUSE1,CAUSE2,CAUSE3,BAGCDC,CHHARNES,CHMAKE,CHORIENT,CHSHIELD,CHTETHER,CHTYPE,DEATH,DVBAG,EJCTAREA,EJCTMED,EJECTION,ENTRAP,BAGFLDAM,BAGFLOPN,HEADREST,HEIGHT,HOSPSTAY,INJSEV,ISS,BAGMAINT,MANAVAIL,MANFAIL,MANUSE,MEDSTA,MEDFACIL,EYEWEAR,OCCMOBIL,BAGAVOTH,BAGDEPOT,PARUSE,FETALDOA,BELTSOU,POSTURE,PREVACC,PSU,ROLE,SEATPERF,SEATPOS,SEATTYPE,SEX,BAGDAMSO,STBACINC,SEATRACK,TREATMNT,WEIGHT,WORKDAYS,OCCRACE,OCETHNIC,CHUSED,ROLLPROT,INTGREST,STORIENT,GLASGOW,BLOOD,BICARB,ALCINV,ATREAT,DAYWEEK,DRGINV,EVENTS,MANCOLL,MONTH,PSUSTRAT,TIME,VEHFORMS,YEAR,ADMINSS,ACCSEQDV,ACCTYPE,ALCTEST,ANGTHIS,ANGOTHER,CLIMATE,BAREQSP,BODYTYPE,CARGOWGT,CONDTREE,CURBWGT,DVEST,DOCTRAJ,DRIVDIST,DRINKING,DRPRES,DRZIP,DVBASIS,DVCONFID,DVLAT,DVLONG,DVTOTAL,ENERGY,ETHNICIT,FOVERIDE,IMPACTSP,INSPTYPE,LANES,LGTCOND,MAKE,MANEUVER,MODEL,MODELYR,OCCFORMS,OCUPANTS,PREEVENT,PREMOVE,PREILOC,PREISTAB,ALIGNMNT,SURCOND,PROFILE,SURTYPE,RACE,RELINTER,ROLINDIR,ROLINLOC,ROLINTYP,ROLLOBJ,PROLLMAN,INTEROLL,ROLLDIST,ROLLOVER,ROVERIDE,SPECOTH,SPLIMIT,TOWHITCH,TOWPAR,TRAFCONT,TRANSTAT,TRCTLFCT,TRAVELSP,TRAFFLOW,TRIPLOC,VEHUSE,VINLNGTH,DRUGS,VINMODYR,MCYCLDS,VEHWGT,VINMAKE,VINAMOD,SERTR,VINBT,ROOF1,ROOF2,ROOF3,ANTILOCK,FRTWHLDR,FOURWHDR,RESTYPE,CARBUR,FUELCODE,WGTCDTR,VEHTYPE,WHLDRWHL,DAYRUNLT,otvehwgt,otbdytyp,VTREAT,ACCSEQ1,ACCSEQ2,ALTVEH,ORIGAVTW,DIRDAMW,DOCCDC,DVC1,DVC2,DVC3,DVC4,DVC5,DVC6,DVD,DVL,DOF1,DOF2,UNDENDW,EXTENT1,EXTENT2,FUELCAP1,FUELCAP2,FUELPRE1,FUELPRE2,FIRE,FIREORIG,FUELDAM1,FUELDAM2,GAD1,GAD2,FUELLOC1,FUELLOC2,FUELTNK1,FUELTNK2,FUELEAK1,FUELEAK2,FUELGT2,OBJCONT1,OBJCONT2,PDOF1,PDOF2,SDVC1,SDVC2,SDVC3,SDVC4,SDVC5,SDVC6,SDVD,SDVL,SHL1,SHL2,SVL1,SVL2,TDD1,TDD2,WHEELBAS,FAILLF,FAILLR,FAILRF,FAILRR,FAILTG,GLIMPBL,GLIMPLF,GLIMPLR,GLIMPOTH,GLIMPRF,GLIMPRR,GLIMPRUF,GLIMPWS,GLOCCBL,GLOCCLF,GLOCCLR,GLOCCOTH,GLOCCRF,GLOCCRR,GLOCCRUF,GLOCCWS,GLPREBL,GLPRELF,GLPRELR,GLPREOTH,GLPRERF,GLPRERR,GLPRERUF,GLPREWS,GLTYPBL,GLTYPLF,GLTYPLR,GLTYPOTH,GLTYPRF,GLTYPRR,GLTYPRUF,GLTYPWS,OPENLF,OPENLR,OPENRF,OPENRR,OPENTG,PASINTEG,ADAPTEQ,COLUMTYP,CDRIR1,INLOC1,INCOMP1,INMAG1,CDRIR2,INLOC2,INCOMP2,INMAG2,CDRIR3,INLOC3,INCOMP3,INMAG3,CDRIR4,INLOC4,INCOMP4,INMAG4,CDRIR5,INLOC5,INCOMP5,INMAG5,CDRIR6,INLOC6,INCOMP6,INMAG6,CDRIR7,INLOC7,INCOMP7,INMAG7,CDRIR8,INLOC8,INCOMP8,INMAG8,CDRIR9,INLOC9,INCOMP9,INMAG9,CDRIR10,INLOC10,INCOMP10,INMAG10,ODOMETER,RDEFLOC,RIMDEF,COLMTELE,COLMTILT,POSTINT
0,16,1,2,7,97,97,1,1,4,0,0,0,7,0,0,0,0,0,0,4.074e-312,997,0,0,0,0,7,7,3,185,0,0,0,1,4,1,4,0,0,7.001,4,1,1,4,8,1,0,1,2,1,1,11,1,1,97,23,5,0,79,97,7,2,0,2.347e-312,4.129e-312,1,0,1,0,2,0,7,2,2,0,3,3,2333,1,2015,0,1,2,nan,998,998,19,13,14,0,0,141,0,1,1,0,1,1.253e+04,1,4,4.074e-312,-13,13,107,2,0,998,3,2,3,2,6,407,2014,2,2,5,14,4,4,3,1,4,2,7,0,1,4,5,31,3,2,3,1,0,0,48,0,1,0,1,0,777,0,6.001,0,17,0,2014,nan,nan,b'JEEP',b'PRT',b'SPO',b'4W',1,nan,nan,2,NaN,NaN,b'W',NaN,b'G',1,b'T',42.01,b'O',998,98,0,1,2,0,151,143,0,0,0,0,0,1,4,1,148,12,0,148,1,1,nan,nan,nan,0,0,0,1,nan,b'F',b'R',nan,nan,nan,nan,1,nan,nan,44,31,0,998,nan,nan,nan,nan,nan,nan,nan,nan,b'D',b'D',b'E',b'E',b'W',b'O',264,0,0,0,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,0,1,1,2,2,0,2,1,0,1,3,2,3,0,2,3,0,1,1,1,1,1,1,nan,0,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,3,0,0,0,3,0
1,17,1,2,7,97,97,1,1,4,0,0,0,7,0,0,0,2.716e-312,0,0,0,997,0,0,0,0,7,7,3,165,0,0,0,1,4,1,4,0,0,7.001,4,1,1,4,8,1,0,1,2,2,1,13,1,1,97,23,3,0,59,97,7,2,0,2.347e-312,4.129e-312,1,0,1,0,2,0,7,2,2,0,3,3,2333,1,2015,0,1,2,nan,998,998,19,13,14,0,0,141,0,1,1,0,1,1.253e+04,1,4,4.074e-312,-13,13,107,2,0,998,3,2,3,2,6,407,2014,2,2,5,14,4,4,3,1,4,2,7,0,1,4,5,31,3,2,3,1,0,0,48,0,1,0,1,0,777,0,6.001,0,17,0,2014,nan,nan,b'JEEP',b'PRT',b'SPO',b'4W',1,nan,nan,2,NaN,NaN,b'W',NaN,b'G',1,b'T',42.01,b'O',998,98,0,1,2,0,151,143,0,0,0,0,0,1,4,1,148,12,0,148,1,1,nan,nan,nan,0,0,0,1,nan,b'F',b'R',nan,nan,nan,nan,1,nan,nan,44,31,0,998,nan,nan,

# exp

In [35]:
exp_column = year_check(exp)

airbag
RECTYPE
bagseat
RECTYPE
childseat
STRATIF
VERSION
RECTYPE
seatloc
VERSION
RECTYPE
POSGUIDE


In [36]:
pd.DataFrame(exp_column['airbag']['RECTYPE'])
pd.DataFrame(exp_column['bagseat']['RECTYPE'])
exp_column['childseat'][['STRATIF','VERSION']]
exp_column['seatloc'][['VERSION' ,'RECTYPE' ,'POSGUIDE']]

,RECTYPE
11,nan
12,nan
13,1
14,1
15,1


,RECTYPE
11,nan
12,nan
13,1
14,1
15,1


,STRATIF,VERSION
11,nan,nan
12,nan,nan
13,1,1
14,1,1
15,1,1


,VERSION,RECTYPE,POSGUIDE
11,nan,nan,nan
12,1,1,1
13,1,1,1
14,1,1,1
15,1,1,1


In [37]:
exp['airbag_11']

,SCASEID,PSU,CASENO,VEHNO,BAGNO,STRATIF,RATWGT,VERSION,BAGLOC,BAGFUNC,BAGTYPE,BAGDEPLY,BAGFAIL,SWITCHTYP,SWTCHSET,BAGFLOPN,BAGFLDAM,BAGDAMAG,BAGDAMSO,PREVACC,BAGMAINT,BAGEVENT,BAGCDC,DVBAG
0,7.6e+08,2,1,1,1,b'H',322.8,24,1,1,4,7,0,0,7,60,60,60,60,9,9,60,60,997
1,7.6e+08,2,1,1,2,b'H',322.8,24,2,1,4,7,0,0,7,60,60,60,60,0,0,0,0,995
2,7.6e+08,2,1,1,3,b'H',322.8,24,5,1,4,7,0,0,7,60,60,60,60,9,9,60,60,997
3,7.6e+08,2,1,1,4,b'H',322.8,24,5,1,4,7,0,0,7,60,60,60,60,0,0,0,0,995
4,7.6e+08,2,1,1,5,b'H',322.8,24,7,1,4,7,0,0,7,60,60,60,60,9,9,60,60,997
5,7.6e+08,2,1,1,6,b'H',322.8,24,7,1,4,7,0,0,7,60,60,60,60,0,0,0,0,995
6,7.6e+08,2,5,1,1,b'K',42.13,24,1,1,1,7,0,0,7,60,60,60,60,1,0,60,60,997
7,7.6e+08,2,5,1,2,b'K',42.13,24,3,1,1,7,0,0,7,60,60,60,60,0,0,0,0,995
8,7.6e+08,2,5,1,3,b'K',42.13,24,5,1,1,7,0,0,7,60,60,60,60,1,0,60,60,997
9,7.6e+08,2,5,1,4,b'K',42.13,24,5,1,1,7,0,0,7,60,60,60,60,0,0,0,0,995


In [38]:
for expand in expand_name:
    print('{}:'.format(expand) + str(len(exp['{}_15'.format(expand)])))

airbag:9104
bagseat:11863
childseat:129
seatloc:8654
tire:12856
tiredmg:13120


In [39]:
same_column(exp)

,SCASEID,PSU,CASENO,VEHNO,BAGNO,STRATIF,RATWGT,VERSION,RECTYPE,SEATPOS,TLOC
airbag,1,1,1,1,1,1,1,1,1,nan,nan
bagseat,1,1,1,1,1,1,1,1,1,nan,nan
childseat,1,1,1,1,nan,1,1,1,1,1,nan
seatloc,1,1,1,1,nan,1,1,1,1,1,nan
tire,1,1,1,1,nan,1,1,1,1,nan,1
tiredmg,1,1,1,1,nan,1,1,1,1,nan,1


In [40]:
for expand in expand_name:
    print('{}:'.format(expand) + str(len(exp['{}_15'.format(expand)])))

airbag:9104
bagseat:11863
childseat:129
seatloc:8654
tire:12856
tiredmg:13120


In [41]:
same_column(exp)

,SCASEID,PSU,CASENO,VEHNO,BAGNO,STRATIF,RATWGT,VERSION,RECTYPE,SEATPOS,TLOC
airbag,1,1,1,1,1,1,1,1,1,nan,nan
bagseat,1,1,1,1,1,1,1,1,1,nan,nan
childseat,1,1,1,1,nan,1,1,1,1,1,nan
seatloc,1,1,1,1,nan,1,1,1,1,1,nan
tire,1,1,1,1,nan,1,1,1,1,nan,1
tiredmg,1,1,1,1,nan,1,1,1,1,nan,1


#  以下挙動確認用

In [42]:
hoge = pd.DataFrame({'key1':[1, 1, 1],
                     'key2':[3,3,3],
                     'key3':[1, 3, 3],
                     'data_x':['a', 'b', 'c']})
hoge.head()
hoge2 = pd.DataFrame({'key1':[1, 1, 3],
                      'key2':[3, 3, 5],
                      'key3':[1, 3, 4],
                      'data_y':['d', 'e', 'f']})
hoge2.head()
hoge3 = pd.DataFrame({'key1':[1],
                      'data':['g']})
hoge3.head()

,key1,key2,key3,data_x
0,1,3,1,a
1,1,3,3,b
2,1,3,3,c


,key1,key2,key3,data_y
0,1,3,1,d
1,1,3,3,e
2,3,5,4,f


,key1,data
0,1,g


In [43]:
pd.merge(hoge, hoge3, on = 'key1').head()

,key1,key2,key3,data_x,data
0,1,3,1,a,g
1,1,3,3,b,g
2,1,3,3,c,g


In [44]:
hoge3 = pd.merge(hoge, hoge2, on = 'key1', how = 'outer')

In [45]:
hoge3.head(10)

,key1,key2_x,key3_x,data_x,key2_y,key3_y,data_y
0,1,3,1,a,3,1,d
1,1,3,1,a,3,3,e
2,1,3,3,b,3,1,d
3,1,3,3,b,3,3,e
4,1,3,3,c,3,1,d
5,1,3,3,c,3,3,e
6,3,nan,nan,NaN,5,4,f


In [46]:
hoge3['key2_x'] = hoge3['key2_x'] + 1

In [47]:
hoge3.head(10)

,key1,key2_x,key3_x,data_x,key2_y,key3_y,data_y
0,1,4,1,a,3,1,d
1,1,4,1,a,3,3,e
2,1,4,3,b,3,1,d
3,1,4,3,b,3,3,e
4,1,4,3,c,3,1,d
5,1,4,3,c,3,3,e
6,3,nan,nan,NaN,5,4,f


In [48]:
hoge['data_x'] = hoge['data_x'].replace('a', 'e')

In [49]:
hoge.head()

,key1,key2,key3,data_x
0,1,3,1,e
1,1,3,3,b
2,1,3,3,c


In [50]:
#pd.merge(hoge, hoge2, on = ['key1', 'key2'], how = 'outer')
#pd.merge(hoge, hoge2, on = ['key1', 'key2'], how = 'inner')
pd.merge(hoge, hoge2, on = ['key1', 'key2', 'key3'], how = 'left')

,key1,key2,key3,data_x,data_y
0,1,3,1,e,d
1,1,3,3,b,e
2,1,3,3,c,e


In [51]:
test = pd.merge(hoge, hoge2, on = 'key1', how = 'inner')
test.head()
test = pd.merge(test, hoge3, on = 'key1', how = 'inner')
test.head(10)

,key1,key2_x,key3_x,data_x,key2_y,key3_y,data_y
0,1,3,1,e,3,1,d
1,1,3,1,e,3,3,e
2,1,3,3,b,3,1,d
3,1,3,3,b,3,3,e
4,1,3,3,c,3,1,d


,key1,key2_x_x,key3_x_x,data_x_x,key2_y_x,key3_y_x,data_y_x,key2_x_y,key3_x_y,data_x_y,key2_y_y,key3_y_y,data_y_y
0,1,3,1,e,3,1,d,4,1,a,3,1,d
1,1,3,1,e,3,1,d,4,1,a,3,3,e
2,1,3,1,e,3,1,d,4,3,b,3,1,d
3,1,3,1,e,3,1,d,4,3,b,3,3,e
4,1,3,1,e,3,1,d,4,3,c,3,1,d
5,1,3,1,e,3,1,d,4,3,c,3,3,e
6,1,3,1,e,3,3,e,4,1,a,3,1,d
7,1,3,1,e,3,3,e,4,1,a,3,3,e
8,1,3,1,e,3,3,e,4,3,b,3,1,d
9,1,3,1,e,3,3,e,4,3,b,3,3,e


In [52]:
hogehoge = pd.DataFrame()
pd.merge(hoge, hogehoge, on = ['key1'])

KeyError: 'key1'